In [2]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 23.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 26.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 37.6 MB/s eta 0:00:00
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)


In [4]:
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.5 MB/s eta 0:00:0000:0100:01


In [6]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.1 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.1/785.1 kB 16.9 MB/s eta 0:00:00a 0:00:01


In [8]:
import pandas as pd
import argparse
from tqdm import tqdm
import json
from src.bm25 import BM25Okapi
from data.bible_utils import bible_helper, build_candidates
from data.data_utils import get_verse_dicts
from src.eval.eval_utils import load_paired_data, load_whole_sermon_data

from src.relic_utils.utils import build_lit_instance, print_results, NUM_SENTS

#SERMON_DF_PATH = '/data/laviniad/sermons-ir/sermoncentral/fsm/'
BIBLE_PATH = '/home/laviniad/projects/religion_in_congress/data/AmericanKJVBible.txt'
FUZZY_CITATION_DICT_PATH = '/data/laviniad/sermons-ir/fuzzy_citation_dict.json'

#parser = argparse.ArgumentParser()
#parser.add_argument('--split', default="train", type=str)
#parser.add_argument('--output_dir', default="/home/laviniad/projects/religion_in_congress/results/whole_sermon/bm25/", type=str)
#parser.add_argument('--num_examples', default=100, type=int)
#args = parser.parse_args()

bible_df = bible_helper(BIBLE_PATH)
verse_text_dict, fuzzy_citation_dict = get_verse_dicts(FUZZY_CITATION_DICT_PATH, BIBLE_PATH)
verse_text_dict = {k: v.lower() for k,v in verse_text_dict.items()}

text_verse_dict = {v: k for k,v in verse_text_dict.items()}

total = 0
submission_data = {}


cands = build_candidates(bible_df)
verse_to_idx = {v.lower(): k for k,v in enumerate(cands)}

tokenized_corpus = [verse.split(" ") for verse in cands]
bm25 = BM25Okapi(tokenized_corpus)


print(f"{'-' * 25} Now evaluating {'-' * 25}")

results = {
    "mean_rank": [],
    "recall@1": [],
    "recall@3": [],
    "recall@5": [],
    "recall@10": [],
    "recall@50": [],
    "recall@100": [],
    "num_candidates": []
}

QUERY = 'The Bible is very clear that God is the one that raises up those in authority. He raised up each of you, all of us. And I believe that God has ordained and allowed each one of us to be brought here for this specific moment.'

#sermon_df, sermon_verse_data, sermon_texts = load_whole_sermon_data(SERMON_DF_PATH, args.split, verse_text_dict, verse_to_idx, args.num_examples, fuzzy_citation_dict)
#full_len = float(len(commentary_verse_df.index))

ranks = []

tokenized_query = QUERY.split(" ")  # just concatenating prefix and suffix

------------------------- Now evaluating -------------------------


In [10]:
quote_id = verse_to_idx[verse_text_dict['daniel 2:21']]
result_verse_scores = bm25.get_scores(tokenized_query)
sorted_scores = dict(sorted(enumerate(result_verse_scores), key=lambda x: x[0], reverse=True))
sorted_score_idx, sorted_score_vals = list(sorted_scores.keys()), list(sorted_scores.values())

gold_rank = sorted_score_idx.index(quote_id) + 1
ranks.append(gold_rank)

results["mean_rank"].extend(ranks)
results["recall@1"].extend([x <= 1 for x in ranks])
results["recall@3"].extend([x <= 3 for x in ranks])
results["recall@5"].extend([x <= 5 for x in ranks])
results["recall@10"].extend([x <= 10 for x in ranks])
results["recall@50"].extend([x <= 50 for x in ranks])
results["recall@100"].extend([x <= 100 for x in ranks])

print_results(results)

print("*" * 30)

print(result_verse_scores)


Results (1 instances):
mean_rank = 9323.0000, recall@1 = 0.0000, recall@3 = 0.0000, recall@5 = 0.0000, recall@10 = 0.0000, recall@50 = 0.0000, recall@100 = 0.0000, num_candidates = nan, 
******************************
[ 0.         24.04597613 12.93599392 ... 26.06557299  6.17914138
 14.5995305 ]


/home/laviniad/miniconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/laviniad/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
